<h1 align="center">KRX Big-Data Contest</h1>

# [ 1 ] Overview

## 1. Sources

### - Basic

 - `[유가증권]일별 시세정보(주문번호-1300-27)` : 2020년 1분기 주식 정보 -> _CSV 형식_
 - `[유가증권]일별 시세정보(주문번호-1300-30)` : 2021년 1분기 주식 정보 -> _CSV 형식_
 - `[유가증권]일별 시세정보(주문번호-1300-33)` : 2022년 1분기 주식 정보 -> _CSV 형식_

### - Extension ( https://kr.investing.com )

 - `환율 추이` : 2020년, 2021년, 2022년 1분기 -> _CSV 형식_
 - `미국 3년 채권수익률` : 2020년, 2021년, 2022년 1분기 -> _CSV 형식_
 - 각종 주식 지표를 계산하기 위한 추가 KRX 추가 데이터
   - 2019년 12월
   - 2020년 12월
   - 2021년 12월

<br><br><br>

## 2. Targets from `.CSV` files (Input)

| 항목 명 | 항목 영어명 | 모델 학습값 여부 |
|:---:|:---:|:---:|
|`거래일자`|TRD_DD|Y|
|`종목코드`|ISU_CD|N|
|`종목명`|ISU_NM|N|
|`시가`|OPNPRC|Y|
|`고가`|HGPRC|Y|
|`저가`|LWPRC|Y|
|`종가`|CLSPRC|Y|
|(누적)`거래량`|ACC_TRDVOL|Y|
|`업종구분`(지수업종코드)|IDX_IND_CD|N|
|`PER`(주가수익률)|PER|Y|
|`상장일`|LIST_DD|N|
|`시가총액`|MKTCAP|Y|

<!-- <br><br><br>

## 3. Results (Output)

| Property | Description |
|:---:|:---:|
|TRD_DD|`거래일자`|
|ISU_CD|`종목코드`|
|ISU_NM|`종목명`|
|OPNPRC|`시가`|
|HGPRC|`고가`|
|LWPRC|`저가`|
|CLSPRC|`종가`|
|ACC_TRDVOL|(누적)`거래량`|
|IDX_IND_CD|`업종구분`(지수업종코드)|
|PER|`PER`(주가수익률)|
|LIST_DD|`상장일`|
|MKTCAP|`시가총액`|
 -->
<br><br><br><hr>

# [ 2 ] Importing Modules

In [1]:
# Data Handlers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Code Libraries
import os
import copy
import abc
import datetime

<br><br><br>

# [ 3 ] Declarations

In [2]:
############################################################################################################

import copy

class Utils:
    """
    데이터 처리를 위해 유용한 기능들을 정의한 클래스입니다.
    """
    @staticmethod
    def generate_int_range(start:int, end:int)->iter:
        """
        start와 end 사이의 정수들을 반환하는 Generator입니다.
        """
        if(start >= end):
            raise SyntaxError(f"{start} must be larger than {end}")
        while start <= end:
            yield start
            start += 1
            
    @staticmethod
    def clone(target:object)->object:
        """
        깊은 복사를 한 인스턴스를 반환합니다.
        """
        return copy.deepcopy(target)
    
    @staticmethod
    def flatten(target:list)->list:
        """
        2차원의 리스트를 1차원 리스트로 변환시켜 반환합니다.
        """
        return [y for x in target for y in x]
    
############################################################################################################

import pandas

class PandasBasedCSVHandler:
    """
    Pandas 모듈을 기반으로 CSV 파일 데이터를 다루는 클래스입니다.
    """
    def __init__(self, handler:pandas):
        self.__handler = handler
        self.__data = dict()
        
    @property
    def handler(self)->pandas:
        """
        주입받은 Pandas 객체를 반환합니다.
        이미 생성된 handler 인스턴스는 대체될 수 없고 반환만 가능합니다.
        """
        return self.__handler
    
    def take_data_from_CSV_file(self, *, data_id:object, filepath:str, encoding:str="utf-8")->object:
        """
        불러올 CSV 파일의 경로를 받아 데이터를 가져오고
        데이터를 식별할 data_id를 받습니다.
        Argument를 반드시 키워드로 명시하여 Parameter에 전달해야 합니다.
        """
        self.__data[data_id] = self.__handler.read_csv(filepath, encoding=encoding)
            
        return self
    
    def get_CSV_data(self)->dict:
        """
        다음 Dictionary 자료구조를 반환합니다.
        key의 타입(자료형)은 정수형으로 의도되었지만 어떤 타입이 들어올지는 자유입니다.
        value는 Pandas 타입의 객체입니다.
        """
        return self.__data
    
    def validate(self, target_properties:list)->object:
        """
        모든 Pandas 데이터가 target_properties에 명시된 속성을 가지고 있는지 확인합니다.
        만일, 속성이 매칭되지 않으면 예외가 발생할 것입니다.
        모든 과정이 성공하면 True를 반환합니다.
        """
        for data_key, _ in self.__data.items():
            self.__data[data_key][target_properties]
        return True

############################################################################################################
import requests
import json
from pandas import json_normalize

class KRXStockCrawler:
    """
    KRX 정보데이터시스템 데이터 크롤링하는 클래스입니다.
    주식 관련 지표들을 계산하기 위한 추가 데이터들을 얻는 것이 목적입니다.
    """
    def __init__(self):
        self.__url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
        self.__headers = "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
        self.__base_data = {
            "bld": "dbms/MDC/STAT/standard/MDCSTAT01701",
            "locale": "ko_KR",
            "param1isuCd_finder_stkisu0_1": "ALL",
            "share": 1,
            "money": 1,
            "csvxls_isNo": "false"
        }
        
    def execute(self, code:str, start_date:int, end_date:int)->dict:
        """
        다음 변수들을 기반으로 크롤링하고 그 결과를 딕셔너리 자료구조 형태로 반환합니다.
           code        : 종목코드
           start_date  : 시작날짜(ex. 20201201)
           end_date    : 종료날짜(ex. 20201231)
        """
        query_data = {
            "isuCd": code,
            "strtDd": start_date,
            "endDd": end_date,
        }
        res = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', data=dict(**self.__base_data,**query_data))
        dict_json = json.loads(res.text)
        return dict_json['output']
    
############################################################################################################
class KRXStockData:
    """
    
    """
    pass
    

############################################################################################################

class DataVisualization:
    pass

############################################################################################################ㅊ

<br><br><br>

# [ 4 ] Data Preprocessing

## 01. 준비 단계

In [3]:
# "종목코드"를 기준으로 각 CSV 데이터들을 식별하는 용도의 자료구조
index_properties:list = dict()

In [4]:
# 데이터 전처리에 필요한 속성들
selected_properties = {
    'krx':["거래일자","종목코드","종목명", "시가", "고가", "저가", "종가", "거래량", "업종구분", "PER", "상장일", "시가총액"],
    'investing.com': ["날짜", "종가", "오픈", "고가", "저가", "변동 %"]
}

## 01-A. 데이터 수집 및 전처리 단계: `KRX에서 기본적으로 제공받은 CSV`

In [5]:
# class<PandasBasedCSVHandler> 인스턴스 생성
csv_handler_krx:PandasBasedCSVHandler = PandasBasedCSVHandler(pd)

### (1) 데이터 경로 및 이름 설정

In [6]:
# CSV 파일 루트 경로
root_dir:str = os.path.join("..", "data")

# CSV 파일 전체 경로 및 이름 형식
filepath_form:str = os.path.join(root_dir, "{0}","{0}_{1}.csv")

# 각 CSV 파일들을 가져오기 위한 프로파일 list<dict[]>
csv_file_profiles : list = [
    {
        "name" : "[유가증권]일별 시세정보(주문번호-1300-27)",
        "date_range" : [202001, 202003]
    },
    {
        "name" : "[유가증권]일별 시세정보(주문번호-1300-30)",
        "date_range" : [202101, 202103]
    },
    {
        "name" : "[유가증권]일별 시세정보(주문번호-1300-33)",
        "date_range" : [202201, 202203]
    }
]

### (2) CSV 파일 데이터 불러오기

In [7]:
# CSV 프로파일 기반으로 class<PandasBasedCSVHandler> 인스턴스에 데이터 병합
for csv_file_info in csv_file_profiles: # 프로파일 요소 기반 iteration 작업
    for date_num in Utils.generate_int_range(csv_file_info["date_range"][0], csv_file_info["date_range"][1]): # Iterator 생성
        csv_handler_krx.take_data_from_CSV_file(
            data_id = date_num, # Year + Month 형식의 정수
            filepath = filepath_form.format(csv_file_info["name"], date_num), # 폴더를 포함한 전체 경로의 파일명
            encoding="cp949" # 파일 인코딩 명시
        )

### (3) 데이터 속성 검증하기

In [8]:
# 검증할 속성 내용들은 [1. 데이터 경로 및 이름 설정]의 selected_properties 변수 참고
csv_handler_krx.validate(selected_properties['krx'])

True

### (4) 데이터 정형화

In [9]:
dataframe_list = csv_handler_krx.get_CSV_data()

 - 날짜 형식 일치시키기

In [10]:
for df_id in dataframe_list:
    dataframe_list[df_id]["거래일자"] = pd.to_datetime(dataframe_list[df_id]['거래일자'].astype('str'))  

#### - **데이터 형식**

```js
{
    202001 : Pandas,
    202002 : Pandas,
    202003 : Pandas,
    202101 : Pandas,
    202102 : Pandas,
    202103 : Pandas,
    202201 : Pandas,
    202202 : Pandas,
    202203 : Pandas
}
```

<br>
<br>
<br>
<br>
<br>

## 01-B. 데이터 수집 및 전처리 단계: `미국 3년 채권수익률`

In [11]:
# class<PandasBasedCSVHandler> 인스턴스 생성
csv_handler_bond:PandasBasedCSVHandler = PandasBasedCSVHandler(pd)

### (1) 데이터 경로 및 이름 설정

In [12]:
# CSV 파일 루트 경로
root_dir:str = os.path.join("..", "data")

# CSV 파일 전체 경로 및 이름 형식
filepath_form:str = os.path.join(root_dir, "{0}","{1}.csv")

# 각 CSV 파일들을 가져오기 위한 프로파일 list<dict[]>
csv_file_profiles : list = [
    {
        "name" : "미국 3년 채권수익률",
        "date_range" : [202001, 202003]
    },
    {
        "name" : "미국 3년 채권수익률",
        "date_range" : [202101, 202103]
    },
    {
        "name" : "미국 3년 채권수익률",
        "date_range" : [202201, 202203]
    }
]

### (2) CSV 파일 데이터 불러오기

In [13]:
# CSV 프로파일 기반으로 class<PandasBasedCSVHandler> 인스턴스에 데이터 병합
for csv_file_info in csv_file_profiles: # 프로파일 요소 기반 iteration 작업
    for date_num in Utils.generate_int_range(csv_file_info["date_range"][0], csv_file_info["date_range"][1]): # Iterator 생성
        csv_handler_bond.take_data_from_CSV_file(
            data_id = date_num, # Year + Month 형식의 정수
            filepath = filepath_form.format(csv_file_info["name"], date_num), # 폴더를 포함한 전체 경로의 파일명
            encoding="cp949" # 파일 인코딩 명시
        )

### (3) 데이터 속성 검증하기

In [14]:
# 검증할 속성 내용들은 [1. 데이터 경로 및 이름 설정]의 selected_properties 변수 참고
csv_handler_bond.validate(selected_properties['investing.com'])

True

### (4) 데이터 정형화

In [15]:
dataframe_list = csv_handler_bond.get_CSV_data()

 - 날짜 형식 일치시키기

In [16]:
for df_id in dataframe_list:
    dataframe_list[df_id]['날짜'] = pd.to_datetime(dataframe_list[df_id]['날짜'], format='%Y년 %m월 %d일')  

 - DataFrame 병합과 데이터 식별을 위해 각 컬럼 이름 바꿔주기

In [17]:
for df_id in dataframe_list:
    column_replacer = ["BOND " + column_name for column_name in selected_properties['investing.com'] if column_name != "날짜"]
    column_replacer.insert(0,"거래일자")
    dataframe_list[df_id].columns = column_replacer

 - 날짜를 기준으로 역순으로 배열하기

In [18]:
for df_id in dataframe_list:
    dataframe_list[df_id] = pd.concat([dataframe_list[df_id].iloc[::-1]], ignore_index=True) 

#### - **데이터 형식**

```js
{
    202001 : Pandas,
    202002 : Pandas,
    202003 : Pandas,
    202101 : Pandas,
    202102 : Pandas,
    202103 : Pandas,
    202201 : Pandas,
    202202 : Pandas,
    202203 : Pandas
}
```

<br>
<br>
<br>
<br>
<br>

## 01-C. 데이터 수집 및 전처리 단계: `환율 추이`

In [19]:
# class<PandasBasedCSVHandler> 인스턴스 생성
csv_handler_exchange_rate:PandasBasedCSVHandler = PandasBasedCSVHandler(pd)

### (1) 데이터 경로 및 이름 설정

In [20]:
# CSV 파일 루트 경로
root_dir:str = os.path.join("..", "data")

# CSV 파일 전체 경로 및 이름 형식
filepath_form:str = os.path.join(root_dir, "{0}","{1}.csv")

# 각 CSV 파일들을 가져오기 위한 프로파일 list<dict[]>
csv_file_profiles : list = [
    {
        "name" : "환율 추이",
        "date_range" : [202001, 202003]
    },
    {
        "name" : "환율 추이",
        "date_range" : [202101, 202103]
    },
    {
        "name" : "환율 추이",
        "date_range" : [202201, 202203]
    }
]

### (2) CSV 파일 데이터 불러오기

In [21]:
# CSV 프로파일 기반으로 class<PandasBasedCSVHandler> 인스턴스에 데이터 병합
for csv_file_info in csv_file_profiles: # 프로파일 요소 기반 iteration 작업
    for date_num in Utils.generate_int_range(csv_file_info["date_range"][0], csv_file_info["date_range"][1]): # Iterator 생성
        csv_handler_exchange_rate.take_data_from_CSV_file(
            data_id = date_num, # Year + Month 형식의 정수
            filepath = filepath_form.format(csv_file_info["name"], date_num), # 폴더를 포함한 전체 경로의 파일명
            encoding="cp949" # 파일 인코딩 명시
        )

### (3) 데이터 속성 검증하기

In [22]:
# 검증할 속성 내용들은 [1. 데이터 경로 및 이름 설정]의 selected_properties 변수 참고
csv_handler_exchange_rate.validate(selected_properties['investing.com'])

True

### (4) 데이터 정형화

In [23]:
dataframe_list = csv_handler_exchange_rate.get_CSV_data()

 - 날짜 형식 일치시키기

In [24]:
for df_id in dataframe_list:
    dataframe_list[df_id]['날짜'] = pd.to_datetime(dataframe_list[df_id]['날짜'], format='%Y년 %m월 %d일')  

 - DataFrame 병합과 데이터 식별을 위해 각 컬럼 이름 바꿔주기

In [25]:
for df_id in dataframe_list:
    column_replacer = ["EX_RATE " + column_name for column_name in selected_properties['investing.com'] if column_name != "날짜"]
    column_replacer.insert(0,"거래일자")
    dataframe_list[df_id].columns = column_replacer

 - 날짜를 기준으로 역순으로 배열하기

In [26]:
for df_id in dataframe_list:
    dataframe_list[df_id] = pd.concat([dataframe_list[df_id].iloc[::-1]], ignore_index=True) 

#### - **데이터 형식**

```js
{
    202001 : Pandas,
    202002 : Pandas,
    202003 : Pandas,
    202101 : Pandas,
    202102 : Pandas,
    202103 : Pandas,
    202201 : Pandas,
    202202 : Pandas,
    202203 : Pandas
}
```

<hr>

## 02. 수집된 데이터 정리 및 주식 관련 지표 추가

### (1) KRX

In [27]:
krx_data = Utils.clone(csv_handler_krx.get_CSV_data())
krx_data[202001][selected_properties['krx']].head()

,거래일자,종목코드,종목명,시가,고가,저가,종가,거래량,업종구분,PER,상장일,시가총액
0,2020-01-02,KR7000020008,동화약품보통주,8340,8400,8290,8400,111305,의약품 제조업,23.01,19760324,234624348000
1,2020-01-03,KR7000020008,동화약품보통주,8400,8450,8290,8360,96437,의약품 제조업,22.9,19760324,233507089200
2,2020-01-06,KR7000020008,동화약품보통주,8290,8330,8120,8180,73230,의약품 제조업,22.41,19760324,228479424600
3,2020-01-07,KR7000020008,동화약품보통주,8200,8280,8090,8160,117904,의약품 제조업,22.36,19760324,227920795200
4,2020-01-08,KR7000020008,동화약품보통주,8170,8170,7830,7930,263246,의약품 제조업,21.73,19760324,221496557100


### (2) Investing (미국 3년 채권수익률)

In [28]:
bond_data = Utils.clone(csv_handler_bond.get_CSV_data())
bond_data[202001].head()

,거래일자,BOND 종가,BOND 오픈,BOND 고가,BOND 저가,BOND 변동 %
0,2020-01-01,1.6086,1.6086,1.6086,1.6086,0.00%
1,2020-01-02,1.5950,1.6140,1.6250,1.5650,-0.85%
2,2020-01-03,1.5457,1.5620,1.5730,1.5240,-3.09%
3,2020-01-05,1.5240,1.5267,1.5267,1.5240,-1.40%
4,2020-01-06,1.5593,1.5270,1.5760,1.5160,2.32%


### (3) Investing (환율)

In [29]:
exchange_rate_data = Utils.clone(csv_handler_exchange_rate.get_CSV_data())
exchange_rate_data[202001].head()

,거래일자,EX_RATE 종가,EX_RATE 오픈,EX_RATE 고가,EX_RATE 저가,EX_RATE 변동 %
0,2020-01-01,"1,154.02","1,155.07","1,155.32","1,154.08",0.00%
1,2020-01-02,"1,157.35","1,155.02","1,161.15","1,153.48",0.29%
2,2020-01-03,"1,164.95","1,157.94","1,168.83","1,155.70",0.66%
3,2020-01-06,"1,166.94","1,165.89","1,172.99","1,165.78",0.17%
4,2020-01-07,"1,167.30","1,167.54","1,168.82","1,163.11",0.03%


#### - 전처리 결과 변수

| Variable Name | Description |
|:---:|:---:|
|`krx_data`| KRX 유가증권 CSV 데이터 모음|
|`bond_data`| Investing.com 미국 3년 채권수익률|
|`exchange_rate_data`| Investing.com 환율 추이|

## 03. 데이터 병합

In [30]:
target_years = [
    list(Utils.generate_int_range(202001, 202003)),
    list(Utils.generate_int_range(202101, 202103)),
    list(Utils.generate_int_range(202201, 202203))
]

In [31]:
total_data = dict()

for year_id in Utils.flatten(target_years):
    exchange_rate_and_bond = exchange_rate_data[year_id].merge(bond_data[year_id])
    krx_and_exchange_rate_and_bond = krx_data[year_id][selected_properties['krx']].merge(exchange_rate_and_bond, on='거래일자')
    total_data[year_id] = krx_and_exchange_rate_and_bond

In [32]:
len(total_data)

9

 - 테스트: **`2020년 01월`** 에 대한 특정 종목을 통한 병합 확인

In [33]:
total_data[202001].loc[total_data[202001]["종목코드"] == "KR7000020008"] # 동화약품보통주

,거래일자,종목코드,종목명,시가,고가,저가,종가,거래량,업종구분,PER,...,EX_RATE 종가,EX_RATE 오픈,EX_RATE 고가,EX_RATE 저가,EX_RATE 변동 %,BOND 종가,BOND 오픈,BOND 고가,BOND 저가,BOND 변동 %
0,2020-01-02,KR7000020008,동화약품보통주,8340,8400,8290,8400,111305,의약품 제조업,23.01,...,"1,157.35","1,155.02","1,161.15","1,153.48",0.29%,1.5950,1.6140,1.6250,1.5650,-0.85%
916,2020-01-03,KR7000020008,동화약품보통주,8400,8450,8290,8360,96437,의약품 제조업,22.9,...,"1,164.95","1,157.94","1,168.83","1,155.70",0.66%,1.5457,1.5620,1.5730,1.5240,-3.09%
1832,2020-01-06,KR7000020008,동화약품보통주,8290,8330,8120,8180,73230,의약품 제조업,22.41,...,"1,166.94","1,165.89","1,172.99","1,165.78",0.17%,1.5593,1.5270,1.5760,1.5160,2.32%
2748,2020-01-07,KR7000020008,동화약품보통주,8200,8280,8090,8160,117904,의약품 제조업,22.36,...,"1,167.30","1,167.54","1,168.82","1,163.11",0.03%,1.5511,1.5620,1.5700,1.5460,-0.53%
3664,2020-01-08,KR7000020008,동화약품보통주,8170,8170,7830,7930,263246,의약품 제조업,21.73,...,"1,162.25","1,168.30","1,179.67","1,160.50",-0.43%,1.6071,1.5080,1.6370,1.4680,3.61%
4580,2020-01-09,KR7000020008,동화약품보통주,8020,8060,7900,7900,50346,의약품 제조업,21.64,...,"1,158.72","1,162.49","1,163.55","1,157.41",-0.30%,1.5964,1.6040,1.6310,1.5910,-0.67%
5496,2020-01-10,KR7000020008,동화약품보통주,7970,8140,7880,8100,77059,의약품 제조업,22.19,...,"1,157.97","1,159.73","1,164.16","1,157.55",-0.06%,1.5857,1.5990,1.6120,1.5750,-0.67%
6412,2020-01-13,KR7000020008,동화약품보통주,8140,8250,8070,8220,91646,의약품 제조업,22.52,...,"1,153.95","1,158.92","1,159.57","1,153.13",-0.35%,1.6018,1.6260,1.6260,1.5880,1.02%
7328,2020-01-14,KR7000020008,동화약품보통주,8240,8240,8070,8140,100901,의약품 제조업,22.3,...,"1,157.20","1,154.95","1,159.66","1,150.30",0.28%,1.5803,1.6020,1.6120,1.5750,-1.34%
8244,2020-01-15,KR7000020008,동화약품보통주,8160,8170,8000,8090,72255,의약품 제조업,22.16,...,"1,157.91","1,158.20","1,162.80","1,155.85",0.06%,1.5643,1.5780,1.5860,1.5590,-1.01%


#### - **병합된 전체 데이터 변수**

| Variable Name | Description |
|:---:|:---:|
| **total_data** | `krx_data` + `bond_data` + `exchange_rate_data`|

#### - **데이터 형식**

```js
{
    202001 : Pandas,
    202002 : Pandas,
    202003 : Pandas,
    202101 : Pandas,
    202102 : Pandas,
    202103 : Pandas,
    202201 : Pandas,
    202202 : Pandas,
    202203 : Pandas
}
```

# [ 5 ] Stock Predictions

In [34]:
# total_data 이 변수를 활용하시면 됩니다.

In [35]:
total_data[202001].head(5)

,거래일자,종목코드,종목명,시가,고가,저가,종가,거래량,업종구분,PER,...,EX_RATE 종가,EX_RATE 오픈,EX_RATE 고가,EX_RATE 저가,EX_RATE 변동 %,BOND 종가,BOND 오픈,BOND 고가,BOND 저가,BOND 변동 %
0,2020-01-02,KR7000020008,동화약품보통주,8340,8400,8290,8400,111305,의약품 제조업,23.01,...,"1,157.35","1,155.02","1,161.15","1,153.48",0.29%,1.595,1.614,1.625,1.565,-0.85%
1,2020-01-02,KR7000040006,KR모터스보통주,288,288,278,282,511750,그외 기타 운송장비 제조업,-,...,"1,157.35","1,155.02","1,161.15","1,153.48",0.29%,1.595,1.614,1.625,1.565,-0.85%
2,2020-01-02,KR7000050005,경방보통주,9280,9600,9270,9580,51436,종합 소매업,12.95,...,"1,157.35","1,155.02","1,161.15","1,153.48",0.29%,1.595,1.614,1.625,1.565,-0.85%
3,2020-01-02,KR7000060004,메리츠화재해상보험보통주,17950,17950,17050,17150,301623,보험업,8.08,...,"1,157.35","1,155.02","1,161.15","1,153.48",0.29%,1.595,1.614,1.625,1.565,-0.85%
4,2020-01-02,KR7000070003,삼양홀딩스보통주,67000,68000,65700,67300,9070,기타 금융업,7.44,...,"1,157.35","1,155.02","1,161.15","1,153.48",0.29%,1.595,1.614,1.625,1.565,-0.85%


In [36]:
total_data[202001].columns

Index(['거래일자', '종목코드', '종목명', '시가', '고가', '저가', '종가', '거래량', '업종구분', 'PER',
       '상장일', '시가총액', 'EX_RATE 종가', 'EX_RATE 오픈', 'EX_RATE 고가', 'EX_RATE 저가',
       'EX_RATE 변동 %', 'BOND 종가', 'BOND 오픈', 'BOND 고가', 'BOND 저가',
       'BOND 변동 %'],
      dtype='object')